In [19]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import ast

# Read the CSV file
df = pd.read_csv('/content/transactions.csv')

# Convert the 'Items' column from string representation of list to actual list
df['Items'] = df['Items'].apply(ast.literal_eval)

# Prepare the data for the apriori algorithm
te = TransactionEncoder()
te_ary = te.fit(df['Items']).transform(df['Items'])
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# Apply the Apriori algorithm
frequent_itemsets = apriori(df_encoded, min_support=0.6, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Recommendation function
def recommend_items(owned_items):
    # Filter rules that apply to the owned items
    matching_rules = rules[rules['antecedents'].apply(lambda x: set(owned_items).issubset(x))]
    # Extract the consequents of the matching rules
    recommendations = set()
    for idx, row in matching_rules.iterrows():
        recommendations = recommendations.union(row['consequents'])

    # Remove items that are already owned
    recommendations = recommendations.difference(owned_items)

    return list(recommendations)

# Example usage
owned_items = ['Laptop', 'Mouse']
print(recommend_items(owned_items))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ValueError: The input DataFrame `df` containing the frequent itemsets is empty.